In [8]:
!pip install dash

In [10]:
!pip install contextily

In [12]:
!pip install mapbox

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.patches import Patch
import pandas as pd
import contextily as ctx
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.lines as mlines
from sklearn.neighbors import NearestNeighbors
from matplotlib.animation import FuncAnimation
import csv
import json
from collections import OrderedDict
import os

In [20]:
cd'/Users/andraniquegreen/dsb-210/Project5-FoodDeserts/misc/'

/Users/andraniquegreen/dsb-210/Project5-FoodDeserts/misc


In [35]:
df = pd.read_csv('lila_geo.csv') #reading in csv file to turn into geoJSON 

In [37]:
df.head() # checking columns

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,centroid
0,48,201,552601,48201552601,5526.01,Census Tract 5526.01,G5020,S,2911911,4263,29.962985,-95.567918,"POLYGON ((1425183.367895747 875674.7728196877,...",POINT (1427116.8341884026 875371.9178676985)
1,48,201,240702,48201240702,2407.02,Census Tract 2407.02,G5020,S,10244569,45432,29.980229,-95.405910,POLYGON ((1440405.2688898814 879771.9527256631...,POINT (1442290.3504877267 877948.32459208)
2,48,201,454302,48201454302,4543.02,Census Tract 4543.02,G5020,S,2751912,1867,29.715345,-95.670479,POLYGON ((1416484.7490622199 846968.1277833888...,POINT (1418097.4024242053 847567.1197641033)
3,48,245,11700,48245011700,117.00,Census Tract 117,G5020,S,4622947,392096,30.085600,-94.103118,POLYGON ((1565656.2946706787 896601.3042808988...,POINT (1567160.0944305696 895468.929591723)
4,48,157,674900,48157674900,6749.00,Census Tract 6749,G5020,S,9531606,577166,29.594329,-95.770743,POLYGON ((1406939.9688523682 834181.7234205762...,POINT (1408763.4456686345 833875.5936048499)


In [41]:
# missing county name for sorting purposes since we have id our county of interest
label_df= pd.read_csv('severe_texas_food_desert.csv')

In [51]:
label_df = label_df[['CensusTract', 'County','LALOWI1_20']]

In [62]:
label_df.head()

,CensusTract,County,LALOWI1_20
0,48001950500,Anderson,1508.45
1,48001950600,Anderson,1773.15
2,48005000400,Angelina,930.88
3,48005000500,Angelina,1356.27
4,48005000600,Angelina,1384.21


In [65]:
df.shape

(965, 14)

In [67]:
label_df.shape

(965, 3)

In [77]:
label_df['GEOID']= label_df['CensusTract']

/var/folders/jy/87nrhv716lq4zwb7ck6sngg80000gn/T/ipykernel_88199/2544757858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_df['GEOID']= label_df['CensusTract']


In [81]:
label_df.drop(columns='CensusTract', inplace=True)

/var/folders/jy/87nrhv716lq4zwb7ck6sngg80000gn/T/ipykernel_88199/2996917451.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_df.drop(columns='CensusTract', inplace=True)


In [83]:
label_df.head()

,County,LALOWI1_20,GEOID
0,Anderson,1508.45,48001950500
1,Anderson,1773.15,48001950600
2,Angelina,930.88,48005000400
3,Angelina,1356.27,48005000500
4,Angelina,1384.21,48005000600


In [95]:
total_df = pd.merge(label_df, df, how='inner', on='GEOID')

In [97]:
total_df = total_df[total_df['County']=='Hidalgo']

In [107]:
total_df.head()

,County,LALOWI1_20,GEOID,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,centroid
537,Hidalgo,1816.77,48215020101,48,215,20101,201.01,Census Tract 201.01,G5020,S,3498526,29110,26.210943,-98.339099,POLYGON ((1164928.1876394637 451719.5900280449...,POINT (1166252.5168108218 452270.79238716484)
538,Hidalgo,2759.38,48215020102,48,215,20102,201.02,Census Tract 201.02,G5020,S,34141368,988830,26.168287,-98.350416,POLYGON ((1161403.456506042 447162.49812601716...,POINT (1165186.0352267835 448528.5505631402)
539,Hidalgo,764.81,48215020201,48,215,20201,202.01,Census Tract 202.01,G5020,S,9373466,0,26.237654,-98.384895,POLYGON ((1160026.117425336 453844.32134431147...,POINT (1161614.8504777164 455383.4633214592)
540,Hidalgo,581.47,48215020202,48,215,20202,202.02,Census Tract 202.02,G5020,S,7347345,0,26.233469,-98.358540,POLYGON ((1162841.0248790286 453599.2738678567...,POINT (1164258.7124164728 454957.97758230905)
541,Hidalgo,1886.75,48215020204,48,215,20204,202.04,Census Tract 202.04,G5020,S,5105420,0,26.236217,-98.334734,POLYGON ((1165323.7797328886 454525.3145418332...,POINT (1166636.205629543 455298.5492465442)


In [116]:
total_df = total_df.sort_values(by=['LALOWI1_20'], ascending=False)

In [130]:
total_df['coordinates'] = total_df['INTPTLON'].astype(str) + ',' + total_df['INTPTLAT'].astype(str)

In [132]:
total_df.head()

,County,LALOWI1_20,GEOID,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,centroid,coordinates
589,Hidalgo,11182.99,48215024112,48,215,24112,241.12,Census Tract 241.12,G5020,S,200931816,15468,26.400457,-98.313570,POLYGON ((1159034.860259485 465693.70420858567...,POINT (1167442.4123919222 473946.9158094257),"-98.3135702,26.4004565"
578,Hidalgo,7660.82,48215023515,48,215,23515,235.15,Census Tract 235.15,G5020,S,32964391,0,26.275813,-98.077555,POLYGON ((1187886.4380680444 458634.2500087055...,POINT (1192292.2292247736 460108.5664044524),"-98.0775554,26.2758125"
544,Hidalgo,7156.30,48215020503,48,215,20503,205.03,Census Tract 205.03,G5020,S,18359389,0,26.159350,-98.266600,POLYGON ((1170672.5698992326 446035.6353138976...,POINT (1173587.2047457325 446874.61692200653),"-98.2665999,26.1593504"
563,Hidalgo,6765.38,48215022104,48,215,22104,221.04,Census Tract 221.04,G5020,S,25118109,42370,26.209176,-98.069596,"POLYGON ((1191622.5863473695 450608.14994138, ...",POINT (1194225.8874546054 452663.5483008781),"-98.0695963,26.2091761"
576,Hidalgo,6635.83,48215023511,48,215,23511,235.11,Census Tract 235.11,G5020,S,50572727,0,26.379152,-98.092324,"POLYGON ((1185307.430771602 469954.3949619273,...",POINT (1190617.5695845888 471546.5544308076),"-98.092324,26.3791517"
